# Как достать признаки с LizaAlert и оформить это в jupyter

Сначала импортируем необходимые пакеты из соответствующих библиотек

In [34]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import scrapy
import nltk
from datetime import datetime, date, time
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/vgodie/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
my_url = 'https://lizaalert.org/forum/viewtopic.php?f=173&t=23959&sid=46a74e963d101e46f5a373d05fee08fb'
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=120&t=2360"
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25112"# boy and AK-47
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25087" #девочка с необычным лицом
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25197" #косоглазый чел
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25104" #ded anisimov
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25098" #ded Medvedev
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25109" #дед Павлов
#my_url = 'https://lizaalert.org/forum/viewtopic.php?f=276&t=25083'#boy Sayakbaev
#my_url = 'https://lizaalert.org/forum/viewtopic.php?f=276&t=25078' #grandma Leandrova
#my_url = "https://lizaalert.org/forum/viewtopic.php?f=276&t=25077" #girl Kamilla
client = uReq(my_url)  # создаем документ, являющийся сырым кодом сайта

In [3]:
page_html = client.read() # функция .read() позволяет вывести на экран все содержимое файла, а если мы это сохраним в переменную, 
                          # то получим объект, который хранит в себе все строки этого файла

In [4]:
client.close()

In [5]:
ps = soup(page_html, "html.parser")

In [6]:
type(ps)

bs4.BeautifulSoup

In [7]:
english_stops = ['.', ',', ':', ';', "!", "?", "-"]

Далее, для работы понадобится открыть код страницы, которую мы хотим парсить. Для этого правой кнопки мыши нажимаем на страницу, затем выбираем просмотреть код, который затем должен появиться в правой стороне страницы.

In [8]:
containers = ps.find_all("div", {"class": "content"})
len(containers)

2

In [9]:
main = ps.find('div', {'class':'content'}).text

In [10]:
posts = ps.find('div', {'class':'pagination'})
posts
re.search(r'\d+', str(posts)).group()

'2'

In [11]:
re.search(r'\d+', str(posts)).group()

'2'

Код для возраста 

In [12]:
title = ps.find("title").extract()
title

<title>Найден.Жив.Кисилев Анатолий Валерьевич, 28 лет, с.Шведино • Форум lizaalert.org</title>

In [13]:
re.search(r'\d+', str(title)).group()

'28'

In [14]:
container = containers[0]
containers[0]

<div class="content"><span style="font-size: 140%; line-height: 116%;"><span style="font-weight: bold">Кисилев Анатолий Валерьевич 28 лет, с. Шведино, Петровский р-он, Ставропольский край </span></span><br/><br/>3 января 2019 ушёл из дома и не вернулся<br/><br/><img alt="Изображение" src="https://a.radikal.ru/a23/1901/db/6bba3a67e9bb.jpg"/><br/><br/><a class="postlink" href="http://www.radikal.ru" rel="nofollow" target="_blank"><img alt="Изображение" src="https://c.radikal.ru/c23/1901/bf/8a5b12f76ed8.jpg"/></a><br/><br/><span style="font-weight: bold">Приметы:</span><br/>Рост: 186 см.<br/>Телосложение: xудощавое<br/>Цвет глаз: серо-голубые<br/>Волосы: русые, короткая стрижка, прямые<br/><br/><span style="font-weight: bold">Был одет:</span> Серое  укороченное приталенное пальто, тёмно-синие джинсы, коричневые тряпочные берцы, серый длинный вязаный шарф, черная шапка.<br/><br/><span style="font-weight: bold">Особые приметы:</span> рост 186 см, худощавого телосложения, волосы светлые, гла

In [15]:
container.img["alt"]  # проверили наличие или отсутствие изображения

'Изображение'

In [16]:
contents = container.contents
contents

[<span style="font-size: 140%; line-height: 116%;"><span style="font-weight: bold">Кисилев Анатолий Валерьевич 28 лет, с. Шведино, Петровский р-он, Ставропольский край </span></span>,
 <br/>,
 <br/>,
 '3 января 2019 ушёл из дома и не вернулся',
 <br/>,
 <br/>,
 <img alt="Изображение" src="https://a.radikal.ru/a23/1901/db/6bba3a67e9bb.jpg"/>,
 <br/>,
 <br/>,
 <a class="postlink" href="http://www.radikal.ru" rel="nofollow" target="_blank"><img alt="Изображение" src="https://c.radikal.ru/c23/1901/bf/8a5b12f76ed8.jpg"/></a>,
 <br/>,
 <br/>,
 <span style="font-weight: bold">Приметы:</span>,
 <br/>,
 'Рост: 186 см.',
 <br/>,
 'Телосложение: xудощавое',
 <br/>,
 'Цвет глаз: серо-голубые',
 <br/>,
 'Волосы: русые, короткая стрижка, прямые',
 <br/>,
 <br/>,
 <span style="font-weight: bold">Был одет:</span>,
 ' Серое  укороченное приталенное пальто, тёмно-синие джинсы, коричневые тряпочные берцы, серый длинный вязаный шарф, черная шапка.',
 <br/>,
 <br/>,
 <span style="font-weight: bold">Особые 

Код, чтобы достать телесложение

In [17]:
for string in contents:
    if "елосложени" in string:
        no_stops_string = ''.join([t for t in string if t not in english_stops])
        tokens = word_tokenize(no_stops_string)
for i in range(len(tokens)):
    if "елосложени" in tokens[i]:
        if tokens[i][-1] == "я":
            body = tokens[i - 1]
        else:
            body = tokens[i + 1]
body.lower()
a = WordNetLemmatizer()
a.lemmatize(body)

'худощавого'

In [18]:
body

'худощавого'

In [19]:
## Нужно доработать код! Или сделать заново
def eyes_color(contents):
    ec = "Не указано"
    for string in contents:
        if "лаз" in string:
            no_stops_string = ''.join([t for t in string if t not in english_stops])
            tokens = word_tokenize(no_stops_string)
            tokens.append("Кто-то объебался")
            for i in range(len(tokens)):
                if "лаз" in tokens[i]:
                    ec = tokens[i + 1]
                if ec == "Кто-то объебался":
                    ec = tokens[i-1]
                    break
    return ec
eyes_color(contents)

'сероголубые'

Код, чтобы достать цвет волос

In [20]:

for string in contents:
    if "олосы" in string:
        no_stops_string_1 = ''.join([t for t in string if t not in english_stops])
        tokens_1 = word_tokenize(no_stops_string_1)
for i in range(len(tokens_1)):
    if "олосы" in tokens_1[i]:
        hair_colour = tokens_1[i + 1]
hair_colour.lower()
b = WordNetLemmatizer()
b.lemmatize(hair_colour)

'светлые'

In [21]:
def hair_color(contents):
    hc = ''
    for string in contents:
        if "олосы" in string:
            no_stops_string_1 = ''.join([t for t in string if t not in english_stops])
            tokens_1 = word_tokenize(no_stops_string_1)
            for i in range(len(tokens_1)):
                if "олосы" in tokens_1[i]:
                    hc = tokens_1[i + 1]
                    break
    return hc

In [22]:
hair_color(contents)

'светлые'

In [23]:
tokens_1

['рост',
 '186',
 'см',
 'худощавого',
 'телосложения',
 'волосы',
 'светлые',
 'глаза',
 'сероголубые',
 'нос',
 'курносый',
 'рыжеватая',
 'щетина']

Сделаем вспомогательные словари для следующего шага

In [32]:
months_length = {"янв": 31, "фев": 28, "мар": 31, "апр": 30, "май": 31, "июн": 30, "июл": 31, "авг": 31, "сен": "30",
                "окт": 31, "ноя": 30, "дек": 31}
months_num = {"янв": 1, "фев": 2, "мар": 3,"апр": 4,"мая": 5, "май": 5,"июн": 6,"июл": 7,"авг": 8,"сен": 9,"окт": 10,"ноя": 11,"дек": 12}

In [25]:
loss_find = {"время пропажи": "", "время нахождения": ""}

Код, чтобы достать время пропажи

In [26]:
def lost_timing(x):
    if re.search(r'\d+ \w+ \d{4}|\d{2}.\d{2}.\d{4}', x).group() == None:
        return "Сбой"
    else:
        lost_time = re.search(r'\d+ \w+ \d{4}|\d{2}.\d{2}.\d{4}', x).group()
        if "." in lost_time:
            lost_time = lost_time[:2] + " " + months_num[lost_time[3:5]]
        elif "я" in lost_time or "т" in lost_time:
            return re.search(r'\d+ \w{3}').group()
    


In [27]:
main

    

'Кисилев Анатолий Валерьевич 28 лет, с. Шведино, Петровский р-он, Ставропольский край 3 января 2019 ушёл из дома и не вернулсяПриметы:Рост: 186 см.Телосложение: xудощавоеЦвет глаз: серо-голубыеВолосы: русые, короткая стрижка, прямыеБыл одет: Серое  укороченное приталенное пальто, тёмно-синие джинсы, коричневые тряпочные берцы, серый длинный вязаный шарф, черная шапка.Особые приметы: рост 186 см, худощавого телосложения, волосы светлые, глаза серо-голубые, нос курносый, рыжеватая щетина.Таблица прозвона больниц и ОВДОриентировка на печатьОриентировка на репост--------------------------------------------------Координатор: Войдан Дмитрий 89620955660Инфорг: Изя Светлана 89624940466'

In [39]:
notmain = ps.find('div', {'class':'notice'}).text

AttributeError: 'NoneType' object has no attribute 'text'

In [28]:
lost_time = lost_time.replace(re.search(r' \w+', lost_time).group(), re.search(r' \w{3}', lost_time).group())
lost_time

NameError: name 'lost_time' is not defined

In [ ]:
loss_find["время пропажи"] = lost_time

Код, чтобы достать время нахождения

In [ ]:
postbody = ps.find('div', {"class":"notice"}).text

In [ ]:
postbody

In [ ]:
find_time = re.search(r'\d{2} \w+', str(postbody)).group()
find_time

In [ ]:
if "0" in find_time:
    find_time = find_time.replace("0", "")

In [ ]:
loss_find["время нахождения"] = find_time

In [ ]:
loss_find

# А теперь реализуем то, ради чего были найдены предыдущие два пункта: время поиска

In [ ]:
ml = list(months_length.keys())
lt = re.search(r'\w{3}', loss_find["время пропажи"]).group()
ft = re.search(r'\w{3}', loss_find["время нахождения"]).group()

In [ ]:
if lt == ft:
    search_time = int(re.search(r'\d+', loss_find["время нахождения"]).group()) - int(re,search(r'\d+', loss_find["время пропажи"]).group())
elif ml.index(ft) > ml.index(lt):
    search_time = int(re.search(r'\d+', loss_find["время нахождения"]).group())
    for i in range(ml.index(ft) - 1, ml.index(lt) - 1, -1):
        if ml[i] != lt:
            search_time += months_length[ml[i]]
        else:
            search_time += months_length[ml[i]] - int(re.search(r'\d+', loss_find["время пропажи"]).group())
elif ml.index(ft) < ml.index(lt):
    search_time = int(re.search(r'\d+', loss_find["время нахождения"]).group())
    i = ml.index(ft) - 1
    while i >= 0:
        if ml[i] != lt:
            search_time += months_length[ml[i]]
        i -= 1
    for j in range(len(ml) - 1, ml.index(lt) - 1, -1):
        if ml[j] != lt:
            search_time += months_length[ml[j]]
        else:
            search_time += months_length[ml[j]] - int(re.search(r'\d+', loss_find["время пропажи"]).group())
        
    

            
    
        
        

In [ ]:
search_time

In [ ]:
containers2 = ps.findAll("div", {"class":"content"})[0].contents
containers2

In [ ]:
containers.contents

Код для доставания возраста 

In [29]:
def lost_timing(x):
    if re.search(r'\d+ \w+ \d{4}|\d{2}.\d{2}.\d{4}', x) == None:
        return "Сбой"
    else:
        lost_time = re.search(r'\d+ \w+ \d{4}|\d{2}.\d{2}.\d{4}', x).group()
        print(lost_time)
        if "я" in lost_time or "т" in lost_time:
            lts = lost_time.split()
            print(lts)
            lts[1] = months_num[lts[1][:3]]
            lts = list(map(int, lts))
            lost_time = date(lts[-1], lts[-2], lts[-3])
            return lost_time
            #lost_time = date(lts[-1], [-2], [-3])
        else:
            lts = lost_time.split('.')
            lts = list(map(int, lts))
            lost_time = date(lts[-1], lts[-2], lts[-3])
            return lost_time

In [35]:
lost_timing(main)

3 января 2019
['3', 'января', '2019']


datetime.date(2019, 1, 3)

In [36]:
def find_timing(x):
    if re.search(r'\d+ \w+ \d{4}|\d{2}.\d{2}.\d{4}', x) == None:
        return "Сбой"
    else:
        find_time = re.search(r'\d+ \w+ \d{4}|\d{2}.\d{2}.\d{4}', x).group()
        print(find_time)
        fts = find_time.split()
        print(fts)
        fts[1] = months_num[fts[1]]
        fts = list(map(int, fts))
        find_time = date(fts[-1], fts[-2], fts[-3])
        return find_time

In [37]:
find_timing(notmain)

NameError: name 'notmain' is not defined